<a href="https://colab.research.google.com/github/sayarghoshroy/LexiNER/blob/main/BE_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Getting content from a set of URLs

In [2]:
# Uncomment if boilerpipe3 does not exist
# !pip install boilerpipe3
from boilerpipe.extract import Extractor
from tqdm import tqdm

In [3]:
import nltk
# To Download punkt module:
# nltk.download('punkt')
import pickle
import sys
import re

In [4]:
url_list = ['https://eisamay.indiatimes.com/nation/let-commander-3-other-terrorists-killed-in-kashmir/articleshow/78599215.cms',
            'https://eisamay.indiatimes.com/nation/india-records-74383-covid-19-cases-tally-crosses-70-lakh-mark/articleshow/78599510.cms',
            'https://eisamay.indiatimes.com/nation/delhi-student-18-beaten-to-death-allegedly-over-relationship-with-girl/articleshow/78589618.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article11',
            'https://eisamay.indiatimes.com/entertainment/cinema/sharpshooter-involved-in-2000-attack-on-rakesh-roshan-arrested/articleshow/78593074.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4',
            'https://eisamay.indiatimes.com/crime/two-men-allegedly-stabbed-tea-shop-worker-after-he-refused-to-give-them-a-cigarette-in-delhi/articleshow/78592505.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article8',
            'https://eisamay.indiatimes.com/entertainment/cinema/corona-positive-bengali-legend-actor-soumitra-chatterjee-is-well-now/articleshow/78587347.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article7',
            'https://eisamay.indiatimes.com/nation/kedarnath-helicopter-service-starts-890-use-chopper-on-first-day/articleshow/78590624.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4',
            'https://eisamay.indiatimes.com/world/5-killed-in-collision-between-plane-microlight-aircraft-in-france/articleshow/78596847.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article11',
            'https://eisamay.indiatimes.com/nation/india-continued-to-report-a-decline-in-the-active-cases-of-the-coronavirus-govt/articleshow/78592214.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article9',
            'https://eisamay.indiatimes.com/nation/ambulance-driver-who-ferried-200-bodies-of-covid-patients-since-march-dies-of-virus-in-delhi/articleshow/78601226.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4']

print("Number of URLs: " + str(len(url_list)))
content_collection = []
# content_collection format
# {'url': <article_URL>, 'content': <boilerpipe_content>}

Number of URLs: 10


In [5]:
failed_extraction_count = 0
failed_URLs = []

for idx in tqdm(range(len(url_list))):
  url = url_list[idx]
  try:
    extractor = Extractor(extractor = 'ArticleExtractor', url = url)
    content = str(extractor.getText())
    # extractor.getText() returns a java lang string
    content_collection.append({'url': url, 'content': content})
  except Exception as e:
    failed_extraction_count += 1
    failed_URLs.append(url)
    print("Error Extracting Page at " + url)

print("")
print("Number of URL Contents Extracted Successfully = " + str(len(url_list) - failed_extraction_count))

100%|██████████| 10/10 [00:11<00:00,  1.17s/it]


Number of URL Contents Extracted Successfully = 10


#### Framing Named Entity Recognition Rules:

- Refer to this [resource for Bengali NER](http://cdn.iiit.ac.in/cdn/ltrc.iiit.ac.in/ner-ssea-08/drafts/1.pdf)


In [6]:
def clean(txt):
  # Cleans Text for Custom Tokenization
  # Removes English Characters and Numbers
  
  txt = txt.replace("\n", " ").replace("\r", " ").replace("|", " | ").replace("।", " । ")
  # In Bengali, । marks the end of a sentence
  punc_list = '!"#$&*+,.-/;?@\^_~)('
  # Removing a set of special characters
  t = str.maketrans(dict.fromkeys(punc_list, " "))
  txt = txt.translate(t)
  # Removing single quotes and backticks
  t = str.maketrans(dict.fromkeys("'`", ""))
  txt = txt.translate(t)
  return txt

def regtok(txt):
  txt = clean(txt)
  regex = re.compile(r'(\d+|\s+|=|}}|\|)')
  tokens = [token.strip() for token in regex.split(txt) if token.strip() != '' and token.strip().isalnum() != True]
  return tokens

In [ ]:
loc_token_end = ['সরনী', 'রোড', 'স্ট্রিট', 'লেন', 'থানা', 'স্কুল', 'বিদ্যালয়', 'কলেজ', 'নদী', 'লেক', 'হ্রদ', 'সাগর', 'মহাসাগর', 'পাহাড়', 'পর্বত']
loc_connect_end = ['নগর', 'গঞ্জ', 'গ্রাম', 'পুর', 'গড়']
person_begin = ['শ্রী', 'শ্রীযুক্ত', 'ডঃ', 'মিঃ', 'মিস', 'মিসেস', 'বেগম', 'বিবি', 'মোঃ', 'ডাক্তার', 'ডক্টর', 'স্বামী', 'সৈয়দ', 'রেভারেন্ড', 'প্রফ', 'অধ্যাপক'] 
person_end = ['বাবু', 'দাদা', 'দা', 'সাহেব', 'কাকু']
person_ref_end = ['রা', 'এরা', 'দের', 'কে']
person_marker = ['ৎ', ' ँ']
ltd_prev_end = ['পাবলিক', 'প্রাইভেট']
limited = 'লিমিটেড'
and_name = 'এণ্ড'
and_end = ['সনস', 'সন', 'ডটার', 'ডটারস', 'ডটার্স ', 'কম্পানি', 'কোম্পানি']
org_end = ['ইনস্টিটিউট', 'এন্টারপ্রাইস', 'কোম্পানি', 'কোম্পানি', 'কর্পোরেশন']

def get_named_entities(text):
  text = clean(text)
  tokens = regtok(text)
  size = len(tokens)
  
  tags = ['none' for idx in range(size)]
  # default tag is 'none'

  for idx in range(size):
    token = tokens[idx]
    if token in loc_connect_end:
      if idx > 0 and tokens[idx - 1] != '|' and tokens[idx - 1] != '।':
        tags[idx - 1] = 'loc'
        tags[idx] = 'loc'
        print("Location Found")

    if idx > 1 and token == limited and tokens[idx - 1] in ltd_prev_end:
      tags[idx - 2] = 'org'
      tags[idx - 1] = 'org'
      tags[idx] = 'org'
      print("Organization Found")

    if idx > 0 and idx < (size - 1) and token == and_name and tokens[idx + 1] in and_end:
      tags[idx - 1] = 'org'
      tags[idx] = 'org'
      tags[idx + 1] = 'org'
      print("Organization Found")

    if token in org_end:
      if idx > 0 and tokens[idx - 1] != '|' and tokens[idx - 1] != '।':
        tags[idx - 1] = 'org'
        tags[idx] = 'org'
        print("Organization Found")

    for init in loc_token_end:
      if token.endswith(init):
        tags[idx] = 'loc'
        print("Location Found")

    for init in person_begin:
      if token.startswith(init):
        print("Person Found")
        tags[idx] = 'per'
        if idx + 1 > size and tokens[idx + 1] != '|' and tokens[idx + 1] != '।':
          tags[idx + 1] = 'per'

    for init in person_end:
      if token.endswith(init):
        tags[idx] = 'per'
        print("Person Found")

    for init in person_ref_end:
      if token.endswith(init):
        tags[idx] = 'per'
        print("Person Found")

    for init in person_marker:
      if init in token:
        tags[idx] = 'per'
        print("Person Found")
  
  return tags

for page in content_collection:
  get_named_entities(str(page['content']))

In [8]:
# ^_^ Thank You